# 第 5 章：格式化输出与让 Claude 开口说话

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格，加载您的 API 密钥并建立`get_completion`辅助函数。

In [ ]:
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# 为预填充文本添加了新参数，默认值为空字符串
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 课程

**Claude 能够以多种方式格式化其输出**。你只需要要求它这样做！

其中一种方式是使用 XML 标签将响应与任何其他多余文本分开。你已经学过可以使用 XML 标签使你的提示对 Claude 来说更清晰、更易于解析。事实证明，你也可以要求 Claude**使用 XML 标签使其输出对人类来说更清晰、更易于理解**。

### 示例

还记得我们在第二章中通过要求 Claude 完全跳过前言来解决的“诗歌前言问题”吗？事实证明，我们也可以通过**告诉 Claude 将诗歌放入 XML 标签中**来达到类似的效果。

In [ ]:
# 变量内容
ANIMAL = "兔子"

# 带有变量内容占位符的提示模板
PROMPT = f"请写一首关于 {ANIMAL} 的俳句。将其放入 <haiku> 标签中。"

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(get_completion(PROMPT))

为什么我们想这样做呢？嗯，将输出放在**XML 标签中，可以让最终用户通过编写一个简短的程序来提取 XML 标签之间的内容，从而可靠地获取诗歌，且仅获取诗歌**。

这种技术的一个扩展是**将第一个 XML 标签放在`assistant`（助手）的回合中**。当你将文本放在`assistant`回合中时，你基本上是在告诉 Claude，Claude 已经说了些什么，并且它应该从那个点继续下去。这种技术被称为“替 Claude 说话”或“预填充 Claude 的响应”。

下面，我们用第一个`<haiku>`XML 标签演示了这一点。请注意 Claude 是如何直接从我们中断的地方继续的。

In [ ]:
# 变量内容
ANIMAL = "猫"

# 带有变量内容占位符的提示模板
PROMPT = f"请写一首关于 {ANIMAL} 的俳句。将其放入 <haiku> 标签中。"

# Claude 响应的预填充内容
PREFILL = "<haiku>"

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print("用户回合：")
print(PROMPT)
print("\n 助手回合：")
print(PREFILL)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude 也擅长使用其他输出格式，特别是`JSON`。如果你想强制输出 JSON 格式（虽然不是完全确定性的，但非常接近），你也可以用开括号`{`来预填充 Claude 的响应。

In [ ]:
# 变量内容
ANIMAL = "猫"

# 带有变量内容占位符的提示模板
PROMPT = f"请写一首关于 {ANIMAL} 的俳句。使用 JSON 格式，键名为 \"first_line\"、\"second_line\" 和 \"third_line\"。"

# Claude 响应的预填充内容
PREFILL = "{"

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n 助手回合")
print(PREFILL)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

下面是一个示例，展示了如何在同一个提示中**使用多个输入变量和输出格式规范，所有这些都通过 XML 标签完成**。

In [ ]:
# 第一个输入变量
EMAIL = "嗨 Zack，只是想快速向你询问一下你本应写好的那个提示的最新进展。"

# 第二个输入变量
ADJECTIVE = "古英语"

# 带有变量内容占位符的提示模板
PROMPT = f"嘿 Claude。这里有一封电子邮件：<email>{EMAIL}</email>。让这封电子邮件更具 {ADJECTIVE} 风格。将新版本写入 <{ADJECTIVE}_email> XML 标签中。"

# Claude 响应的预填充内容（现在是带有变量的 f-string）
PREFILL = f"<{ADJECTIVE}_email>"

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n 助手回合")
print(PREFILL)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### 额外课程

如果你通过 API 调用 Claude，你可以将结束 XML 标签传递给`stop_sequences`参数，让 Claude 在发出你想要的标签后立即停止采样。这可以通过消除 Claude 在给出你关心的答案后可能出现的结束语，从而节省金钱和“到最后一个 token 的时间”（time-to-last-token）。

如果你想在不改变上方任何内容的情况下，尝试本课程的提示，请滚动到课程笔记本的最底部，访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 5.1 - 斯蒂芬·库里 GOAT](#exercise-51---steph-curry-goat)
- [练习 5.2 - 两首俳句](#exercise-52---two-haikus)
- [练习 5.3 - 两首俳句，两种动物](#exercise-53---two-haikus-two-animals)


### 练习 5.1 - 斯蒂芬·库里 GOAT
在被迫做出选择时，Claude 将迈克尔·乔丹指定为史上最伟大的篮球运动员。我们能否让 Claude 选择其他人呢？

修改`PREFILL`变量，**强制 Claude 详细论证斯蒂芬·库里是史上最伟大的篮球运动员**。尽量不要修改`PREFILL`以外的任何内容，因为这是本次练习的重点。

In [ ]:
# 带有变量内容占位符的提示模板
PROMPT = f"谁是史上最伟大的篮球运动员？请选择一位具体的球员。"

# Claude 响应的预填充内容
PREFILL = ""

# 获取 Claude 的响应
response = get_completion(PROMPT, prefill=PREFILL)

# 评估练习正确性的函数
def grade_exercise(text):
    return bool(re.search("勇士", text))

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n 助手回合")
print(PREFILL)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("本练习已正确解决：", grade_exercise(response))

❓ 如果你需要提示，运行下面的单元格！

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### 练习 5.2 - 两首俳句
修改下面的`PROMPT`，使用 XML 标签，让 Claude 写两首关于这种动物的俳句，而不是一首。应该清楚地标明一首诗的结束和另一首诗的开始。

In [ ]:
# 变量内容
ANIMAL = "猫"

# 带有变量内容占位符的提示模板
PROMPT = f"请写一首关于 {ANIMAL} 的俳句。将其放入 <haiku> 标签中。"

# Claude 响应的预填充内容
PREFILL = "<haiku>"

# 获取 Claude 的响应
response = get_completion(PROMPT, prefill=PREFILL)

# 评估练习正确性的函数
def grade_exercise(text):
    return bool(
        (re.search("猫", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n 助手回合")
print(PREFILL)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("本练习已正确解决：", grade_exercise(response))

❓ 如果你需要提示，运行下面的单元格！

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### 练习 5.3 - 两首俳句，两种动物
修改下面的`PROMPT`，使**Claude 生成两首关于两种不同动物的俳句**。使用`{ANIMAL1}`作为第一个替换的占位符，使用`{ANIMAL2}`作为第二个替换的占位符。

In [ ]:
# 第一个输入变量
ANIMAL1 = "猫"

# 第二个输入变量
ANIMAL2 = "狗"

# 带有变量内容占位符的提示模板
PROMPT = f"请写一首关于 {ANIMAL1} 的俳句。将其放入 <haiku> 标签中。"

# 获取 Claude 的响应
response = get_completion(PROMPT)

# 评估练习正确性的函数
def grade_exercise(text):
    return bool(re.search("尾巴", text.lower()) and re.search("猫", text.lower()) and re.search("<haiku>", text))

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print("用户回合")
print(PROMPT)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("本练习已正确解决：", grade_exercise(response))

❓ 如果你需要提示，运行下面的单元格！

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### 恭喜！

如果你已经解决了到目前为止的所有练习，你就可以进入下一章了。祝你提示愉快！

---

# Chapter 5: Formatting Output and Speaking for Claude

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# New argument added for prefill text, with a default value of an empty string
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

**Claude can format its output in a wide variety of ways**. You just need to ask for it to do so!

One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to Claude. It turns out, you can also ask Claude to **use XML tags to make its output clearer and more easily understandable** to humans.

### Examples

Remember the 'poem preamble problem' we solved in Chapter 2 by asking Claude to skip the preamble entirely? It turns out we can also achieve a similar outcome by **telling Claude to put the poem in XML tags**.

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

Why is this something we'd want to do? Well, having the output in **XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags**.

An extension of this technique is to **put the first XML tag in the `assistant` turn**. When you put text in the `assistant` turn, you're basically telling Claude that Claude has already said something, and that it should continue from that point onward. This technique is called "speaking for Claude" or "prefilling Claude's response."

Below, we've done this with the first `<haiku>` XML tag. Notice how Claude continues directly from where we left off.

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude also excels at using other output formatting styles, notably `JSON`. If you want to enforce JSON output (not deterministically, but close to it), you can also prefill Claude's response with the opening bracket, `{`}.

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Below is an example of **multiple input variables in the same prompt AND output formatting specification, all done using XML tags**.

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### Bonus lesson

If you are calling Claude through the API, you can pass the closing XML tag to the `stop_sequences` parameter to get Claude to stop sampling once it emits your desired tag. This can save money and time-to-last-token by eliminating Claude's concluding remarks after it's already given you the answer you care about.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT
Forced to make a choice, Claude designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?

Change the `PREFILL` variable to **compell Claude to make a detailed argument that the best basketball player of all time is Stephen Curry**. Try not to change anything except `PREFILL` as that is the focus of this exercise.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = ""

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### Exercise 5.2 - Two Haikus
Modify the `PROMPT` below using XML tags so that Claude writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [ ]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### Exercise 5.3 - Two Haikus, Two Animals
Modify the `PROMPT` below so that **Claude produces two haikus about two different animals**. Use `{ANIMAL1}` as a stand-in for the first substitution, and `{ANIMAL2}` as a stand-in for the second substitution.

In [ ]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))